<a href="https://colab.research.google.com/github/pavi-ninjaac/NuralNetwork_scratch/blob/main/CNN/ConvolutionalLayer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# conolutional layer
import numpy as np

In [90]:
class con2D:
  def __init__(self, filter , kernal_size, img_channel,strides = (1,1) ):
    self.filter = None
    self.bias = None
    self.input = None
    """Convolute the image with the filter and strids"""
    
    
    f_channel = img_channel               #filter channel must match with the image channel
    self.no_filters = filter                   # number of filters
    f_dim , _ = kernal_size
    self.f_dim =  f_dim              # get the filter size
    self.strides , _ = strides
    f_size = (self.no_filters , f_channel , f_dim , f_dim) # get the strides dimentions

    #initialize the filter
    """ Initalize the filter with the noraml distribution with the standard deviation inversely proportional the square root of the number of units"""
    sdv = 1.0 / np.sqrt(np.prod(f_size))
    self.filter = np.random.normal(loc = 0, scale = sdv, size = f_size)
    #self.filter = np.array([[[1,0,1] , [2,1,1] , [0,1,2]]])
    self.bias = np.zeros((self.no_filters , 1)) # each for the one filters    

  
  #forward propogaton
  def forward_propogation(self, image):
    self.input = image
    (img_channel , input_dim , _ ) = image.shape #input dimensions channel first dimention 
    
    #calculate the output dimension, so according to that i can create the empty array to store value
    out_dim = int(((input_dim - self.f_dim) / self.strides) + 1)
    print('out put dimention',out_dim)
    #create the empty array with output dimetions
    self.output = np.zeros((1 , out_dim, out_dim )) #syntax np.zeros(channel , row , col)
    for curr_filter in range(self.no_filters): # for loop over the number of filters
      curr_y = out_y =0
      
      #move vertically over the image
      while (curr_y + self.f_dim) <= input_dim:
        curr_x = out_x = 0
        #move horizondally
        while (curr_x + self.f_dim) <= input_dim:
          
          """perform convolution operation on the image"""
          self.output[curr_filter , curr_y , curr_x] = np.sum(self.filter[curr_filter] * image[: , curr_y : curr_y + self.f_dim , curr_x : curr_x + self.f_dim]) #+ self.bias[curr_filter]

          curr_x += self.strides
          out_x +=1
        curr_y += self.strides
        out_y += 1
    print(self.output)
    return self.output

  def back_propogation(self , error_input , learning_rate):
      """update the filter """ # gonna use the self.no_filters and self.f_dim
      image_channel , img_dim , _ = self.input.shape

      #initialize the derivations by creating zero array
      dout = np.zeros(self.input.shape)
      d_filter = np.zeros(self.filter.shape)
      d_bias = np.zeros(self.bias.shape)
      for curr_filter in range(self.no_filters): # for loop over the number of filters
        curr_y = out_y =0
      
        #move vertically over the image
        while (curr_y + self.f_dim) <= img_dim:
          curr_x = out_x = 0
          #move horizondally
          while (curr_x + self.f_dim) <= img_dim:
          
            """loss gradiant on filters"""
            #update the filter
            print('filter' , self.filter)
            print('zero filter' , d_filter)
            print(d_filter[curr_filter])
            print(error_input[curr_filter , out_y , out_x] * self.input[: , curr_y : curr_y + self.f_dim , curr_x : curr_x + self.f_dim])
            d_filter[curr_filter] +=  error_input[curr_filter , out_y , out_x] * self.input[: , curr_y : curr_y + self.f_dim , curr_x : curr_x + self.f_dim]

            #gradiant of the input 
            dout[: , curr_y : curr_y + self.f_dim , curr_x : curr_x + self.f_dim] += error_input[curr_filter , out_y , out_x] * self.filter[curr_filter]

            
            curr_x += self.strides
            out_x +=1
          curr_y += self.strides
          out_y += 1
        d_bias[curr_filter] = np.sum(error_input[curr_filter])
    #update the weights
        self.filter = d_filter
        self.bias = d_bias
      print(dout)
      return dout







In [86]:
class MaxPooling2D(con2D):
  def __init__(self , pool_size = 1, strides=1):
    self.pool_size = pool_size #getting the kernal size for pooling the layer out
    self.strides = strides
    self.image = None
    
  def forward_propogation(self , image):

    """ downsample of the image using the kernal size and strides"""
    self.image = image
    img_channel , h_prev , w_prev  = image.shape    # shape of the image
    # calculate output dimensions after the maxpooling operation.
    h = int((h_prev - self.pool_size)/self.strides)+1 
    w = int((w_prev - self.pool_size)/self.strides)+1
    
    # create a matrix to hold the values of the maxpooling operation.
    self.output = np.zeros((img_channel, h, w))
    # slide the window over every part of the image using stride s. Take the maximum value at each step.
    for i in range(img_channel):
        curr_y = out_y = 0
        # slide the max pooling window vertically across the image
        while curr_y + self.pool_size <= h_prev:
            curr_x = out_x = 0
            # slide the max pooling window horizontally across the image
            while curr_x + self.pool_size <= w_prev:
                # choose the maximum value within the window at each step and store it to the output matrix
                self.output[i, out_y, out_x] = np.max(image[i, curr_y:curr_y+self.pool_size, curr_x:curr_x+self.pool_size])
                curr_x += self.strides
                out_x += 1
            curr_y += self.strides
            out_y += 1
    print(self.output)
    return self.output

  def get_max_index(self , arr):
      """return the index of the maximum value in the array"""
      flat_index = np.nanargmax(arr)
      idex = np.unravel_index(flat_index , arr.shape)
      return idex


  def back_propogation(self, output , learning_rate):
      """put the max value in it's old position ,,,fill remaining with zeros"""
      img_channel , img_dim , _  = self.image.shape

      #create an zero value array with the original image size
      dout = np.zeros(self.image.shape)
      # slide the window over every part of the image using stride s. Take the maximum value at each step.
      for cuur_channel in range(img_channel):
        curr_y = out_y = 0
        # slide the max pooling window vertically across the image
        while curr_y + self.pool_size <= img_dim:
            curr_x = out_x = 0
            # slide the max pooling window horizontally across the image
            while curr_x + self.pool_size <= img_dim:
              #obtain the index of the maximum value in that window size
              a,b = self.get_max_index(self.image[cuur_channel , curr_y:curr_y+self.pool_size, curr_x:curr_x+self.pool_size])
              dout[cuur_channel , curr_y+a , curr_x+b] = output[cuur_channel , curr_y , curr_x]

              curr_x += self.strides
              out_x += 1
            curr_y += self.strides
            curr_y += 1
      print(dout)
      return dout






In [4]:
class Flatten(con2D):
  """ get the strignt single channel fully connet layer """
  def __init__(self):
    self.image = None
    self.flatten_image = None
    
  def forward_propogation(self , image):
    #store the image  size to use in the backpropogation
    self.image = image
    image_channel , input_dim , _ = image.shape
    a = image.flatten()  #flatten the image 
    self.flatten_image = np.array([a]) #make it to 2D
    print(self.flatten_image)
    return self.flatten_image 

  def back_propogation(self , output , learning_rate):
    output = output.reshape(self.image.shape)
    print(output)
    return output  # mae it into the original size
    
      

In [5]:
class FClayer(Flatten):
  def __init__(self , input_size , output_size):
    self.input = None
    self.input_size = input_size
    self.output = None
    # initialize the weights using the normal distribution
    self.weights = np.random.rand(self.input_size , output_size) - 0.5
    self.bias =  np.random.rand(1,output_size) - 0.5 
  
  def forward_propogation(self , input):
    self.input = input
    self.output = np.dot(self.input , self.weights) + self.bias
    print(self.output)
    return self.output
  
  def back_propogation(self, output_error , learning_rate):
    print('coming gradiant' , output_error)
    print(' weight' , self.weights)
    print('back input ',self.input)
    print('bias',self.bias)
    print('finding the input error')
    print('the weight transpose' , self.weights.T)
    print('error',output_error ,'*' ,self.weights.T )
    input_error = np.dot(output_error , self.weights.T)  #dE/dX = W.T * dE / dY
    
    
    print('input transpose',self.input.T)
    weights_error = np.dot(self.input.T , output_error) #dE/dW = X.T * dE / dY
    bias_error = output_error                           #dE/dB =  dE / dY
    #update the weights and bias
    self.weights -= learning_rate *  weights_error
    self.bias -= learning_rate * bias_error
    print(input_error)
    return input_error


In [6]:
def softmax_funtion(output):
  """ pass the raw predicted data to the activation function to get the probability"""
  out = np.exp(output) # exponent the predicted output
  return out / np.sum(out)

def softmax_derivated(loss_derivated_error):
  return loss_derivated_error


In [7]:
def catogarical_cross_entropy(prob , label):
  """ calculate the categorical cross entropy loss"""
  return -np.sum(label * np.log(prob)) #multiply the output with the log of probability

def catogarical_cross_entropy_derivated(prob , label):
  loss_derivated_error = prob / label
  print('loss derivated',loss_derivated_error)
  return loss_derivated_error

In [8]:
class softmax(FClayer):
  def __init__(self, actvation = softmax_funtion , activation_derivated = softmax_derivated): #default parameter setting
    self.actvation = actvation #show the activation from the outside class
    self.activation_derivated = activation_derivated

  def forward_propogation(self , input_data):
    self.input = input_data 
    self.output = self.actvation(self.input)
    print('soft max activation',self.output)
    return self.output

  def back_propogation(self ,  output_error , learning_rate):
    return self.activation_derivated(self.input) * output_error

In [17]:
class Relu(con2D):
  """ pass through the relu activation function"""
  def __init__(self):
    self.output = None

  def forward_propogation(self,input):
    input[input<=0] = 0
    self.output = input
    print(self.output)
    return self.output
  
  def back_propogation(self,output_error , learning_rate):
    output_error[output_error<=0] = 0
    return output_error


In [91]:
class network:
  def __init__(self):
    self.layers = []

  #add the layer in the model
  def add(self , layer):
    self.layers.append(layer)
  #set the loss function to use it out
  def setloss(self, loss , loss_derivated):
    self.loss = loss
    self.loss_derivated = loss_derivated

  def fit(self , x_train ,  y_train , epoches ,learning_rate):
    sampels = len(x_train)
    #loop over for epoches
    for i in range(epoches):
      err = 0
      for j in range(sampels):
        print('sample' , j)
        output = x_train[j] 
        for layer in self.layers:
          print(layer)
          output = layer.forward_propogation(output)

        #compute the loss
        err += self.loss(y_train[j] , output)
        print('loss ',err)
        #backpropogation
        error = self.loss_derivated(y_train[j] , output)
        
        #update the weights
        print('backporopogation stated')
        for layer in reversed(self.layers):
          print('back',layer)
          error = layer.back_propogation(error , learning_rate)
      #displae the error and epoches for the reference
      err/=sampels
      print(err)
    print(output)



In [92]:
#build the model 
model = network()
model.add(con2D(filter = 1 , kernal_size =(3,3) , strides = (1,1) , img_channel =1))
model.add(Relu())
model.add(MaxPooling2D(pool_size = 2))
model.add(Relu())
model.add(Flatten())
model.add(FClayer(4,3))
model.add(FClayer(3,2))
model.add(softmax())

In [93]:
i = np.array([[[[1,2,1,0,1] , [2,1,1,2,1] , [3,2,1,0,0] , [0,1,2,3,1] , [2,1,1,0,2]]]])
print(i.shape)
#set the loss to compute the loss
model.setloss(loss = catogarical_cross_entropy , loss_derivated = catogarical_cross_entropy_derivated)
model.fit(i ,[1,0] ,1 , learning_rate = 0.01)

(1, 1, 5, 5)
sample 0
out put dimention 3
[[[1.45532157 1.9195338  0.69920107]
  [2.15873956 1.45775738 0.98596288]
  [2.14337815 1.96910697 1.5599534 ]]]
[[[1.45532157 1.9195338  0.69920107]
  [2.15873956 1.45775738 0.98596288]
  [2.14337815 1.96910697 1.5599534 ]]]
[[[2.15873956 1.9195338 ]
  [2.15873956 1.96910697]]]
[[[2.15873956 1.9195338 ]
  [2.15873956 1.96910697]]]
[[2.15873956 1.9195338  2.15873956 1.96910697]]
[[-0.52070406 -0.1216365   2.41217346]]
[[ 0.13154152 -0.33334135]]
soft max activation [[0.61417189 0.38582811]]
loss  0.0
loss derivated [[1.62820868 2.59182774]]
backporopogation stated
back <__main__.softmax object at 0x7f25c6a9a750>
back <__main__.FClayer object at 0x7f25c6a9af50>
coming gradiant [[ 0.21417705 -0.86396337]]
 weight [[-0.46927596 -0.33909704]
 [ 0.08598564  0.10928827]
 [ 0.09664921 -0.11697013]]
back input  [[-0.52070406 -0.1216365   2.41217346]]
bias [[-0.33548804 -0.21446486]]
finding the input error
the weight transpose [[-0.46927596  0.08598564

In [ ]:
f_size = (2,3,3)
sdv = 1.0 / np.sqrt(np.prod(f_size))
sd = np.random.normal(loc = 0, scale = sdv, size = f_size)
sd

In [ ]:
s = np.array([[[1,0,1] , [2,1,1] , [0,1,2]]])
s

In [ ]:
i = np.array([ [[1,2,1,0,-1] , [-2,1,1,2,1] , [3,2,1,0,0] , [0,1,2,3,1] , [2,1,1,0,2]]])
i[i<=0] = 0
i

In [ ]:
class a:
   
  def __init__(self):
    self.i = 2
  def d(self):
    self.i = 10
    print(self.i)
  def geti(self):
    return self.i
a().d()  
class b(a):
  print(super().geti())

10


RuntimeError: ignored

In [ ]:
s= np.array([[-0.06959409 , 0.12889191],
 [-0.17250638 ,-0.45558094],
 [ 0.05320838 ,-0.37098399]])
s.T

array([[-0.06959409, -0.17250638,  0.05320838],
       [ 0.12889191, -0.45558094, -0.37098399]])

In [ ]:
a= [17., 16., 17., 16.]
s= np.array([a])
s.T

array([[17.],
       [16.],
       [17.],
       [16.]])